In [1]:
import os


In [2]:
%pwd


'c:\\Users\\lenovo\\Desktop\\Emotion-Detection\\research'

In [3]:

os.chdir("../")

In [5]:
%pwd

'c:\\Users\\lenovo\\Desktop\\Emotion-Detection'

In [13]:
from dataclasses import dataclass
from pathlib import Path

# entity--> This is a return type of function for next componenent
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    push_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
from src.Emotion_Detection.constants import *
from src.Emotion_Detection.utils.common import read_yaml,create_directories

In [15]:
# Defining Paths basically we are mergging our constants from config.yaml with entity dataingestion entity
class ConfigurationManager:
    def __init__( self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            push_URL=config.push_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
# it will return all the paths 
        return data_ingestion_config   
      

In [16]:
from src.Emotion_Detection.constants.database import COLLECTION_NAME
from src.Emotion_Detection.data_access.ImageData import Imagedata


In [20]:
import os
import zipfile
from src.Emotion_Detection import logger
from src.Emotion_Detection.utils.common import get_size
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            push_URL = Path(self.config.push_URL)
            zip_download_dir = self.config.root_dir
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from Mongo DB into file {zip_download_dir}")

            image_data=Imagedata()
            zip_path=image_data.download_zipped_image_files(collection_name=COLLECTION_NAME,directory_path=zip_download_dir)
            print(zip_path)


            logger.info(f"Downloaded data from Mongo-DB into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-01 11:58:06,640: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-01 11:58:06,650: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-01 11:58:06,650: INFO: common: created directory at: artifacts]
[2024-05-01 11:58:06,658: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-01 11:58:06,658: INFO: 3337591719: Downloading data from Mongo DB into file artifacts/data_ingestion]
artifacts/data_ingestion\data.zip
[2024-05-01 11:58:08,292: INFO: 3337591719: Downloaded data from Mongo-DB into file artifacts/data_ingestion]
